In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
import numpy as np 
import pandas as pd


In [3]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v13.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v13.csv")



In [4]:
# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]
# df = challenge_set_updated.sample(frac=0.001)

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']



In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Handle categorical columns by Label Encoding
label_encoder = LabelEncoder()

# Specify the columns that are categorical
categorical_cols = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_season', 
                    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction', 
                    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight']

# Convert these categorical columns into numerical form using Label Encoding
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Now, you can proceed with training LightGBM with this processed data


In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
import joblib
import numpy as np
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


# Assuming X_test is your input DataFrame and 'cat_feature_30' is your categorical feature
# Convert categorical feature to string if it's currently in a numeric form
if 'cat_feature_30' in X_test.columns:
    X_test['cat_feature_30'] = X_test['cat_feature_30'].apply(lambda x: str(x) if not pd.isna(x) else 'missing')

# Now you can safely load your model and make predictions
catboost_model = CatBoostRegressor()
catboost_model.load_model("models/catboost_all.cbm")
# Load the pre-trained CatBoost model
#catboost_model = CatBoostRegressor()
#catboost_model.load_model("models/catboost_all.cbm")

# Load the pre-trained XGBoost model
xgboost_model = joblib.load("models/trained_model_20241008_174426.joblib")

# Ensure X_test and y_test are defined and properly preprocessed before this point
# Make predictions with each model
catboost_preds = catboost_model.predict(X_test)
xgboost_preds = xgboost_model.predict(X_test)

# Ensemble technique: Average predictions
ensemble_preds = (catboost_preds + xgboost_preds) / 2

# Evaluate the ensemble model using RMSE
rmse = np.sqrt(mean_squared_error(y_test, ensemble_preds))
print(f'Ensemble Model RMSE: {rmse:.4f}')

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=30]=235.89542477045256 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
# Create an Optuna study object and optimize
study = optuna.create_study(direction='minimize')  # 'minimize' for minimizing RMSE
study.optimize(objective, n_trials=100)

# Print the best trial and its hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value (RMSE): {trial.value}")
print(f"  Params: {trial.params}")


In [ ]:
import joblib


model_final = global_model_pipeline

# Save the ensemble model to a file
model_filename = 'ensemble_model.pkl'
joblib.dump(model_final, model_filename)

print(f"Model saved to {model_filename}")


In [ ]:
from sklearn.preprocessing import LabelEncoder

submission_set_features = submission_set_updated.iloc[:,:-1]

# Now you can use the model to make predictions
submission_set['tow'] = model_final.predict(submission_set_features)
print(submission_set)


In [ ]:
import os
from datetime import datetime

# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Define a timestamp for the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")

# Assuming submission_set is a DataFrame, save it to CSV
submission_set.to_csv(submission_file, index=False)

print(f"Submission saved to {submission_file}")
